In [1]:
! pip install PyPDF2
!pip install langchain
!pip install openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
import PyPDF2
sample_pdf = open(r'D:\University\Test\6.pdf', mode='rb')
pdfdoc = PyPDF2.PdfReader(sample_pdf)

In [3]:
#print(pdfdoc.metadata)

In [4]:
#len(pdfdoc.pages)

In [5]:
#pdfdoc.pages[15].extract_text().replace("\n", " ")

In [6]:
doc=[]
for x in range(len(pdfdoc.pages)):
  page_one=pdfdoc.pages[x]
  text=page_one.extract_text().replace("\n", " ")
  doc.append(text)

In [7]:
import re

def split_text_on_concept(text, max_length):
    """
    Splits the input text into parts with each part having less than max_length characters,
    ensuring each part ends at a complete concept (sentence).

    Parameters:
    text (str): The input text to split.
    max_length (int): The maximum length of each part before splitting at the nearest sentence end.

    Returns:
    list: A list containing the split parts of the text.
    """
    parts = []
    start = 0

    while start < len(text):
        # Determine the end index for the next part
        end = start + max_length
        
        # Adjust the end index to find the nearest sentence boundary (e.g., period, exclamation mark, question mark)
        if end < len(text):
            match = re.search(r'[.!?]', text[end:])
            if match:
                end += match.end()
            else:
                end = len(text)  # If no sentence boundary is found, take the rest of the text
        
        # Append the part to the list
        parts.append(text[start:end].strip())
        
        # Move the start index for the next part
        start = end

    return parts


In [8]:
splitted_docs=[]
for x in doc:
    splitted_doc = split_text_on_concept(x, max_length=500)
    splitted_docs.append(splitted_doc)

In [9]:
splitted_docs[2][0]

'6 D. Cavallucci and N. Khomenko                                             “There is a need for a new approach to  address inventive challenges in design  that on one side fundamentally differs  from widely established optimisation  approaches and on the other side is complimentary to those approaches.”   The next two paragraphs will be dedicated to  introducing two of the major difficulties for  this approach in inventive design: artefact representation and knowledge codification.'

In [10]:
len(splitted_docs)

18

In [11]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [12]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [13]:
Human_prompt= '''
Design methodologies are systematic approaches to resolving design problems. 
They have various tools and concept to assist designers through the design process.
Each tool or concept in every design methodology must have one of the following functions:
1-Some tools help designers better understand design problems.
2-some assist designers in gathering information and improving their knowledge.
3-Some tools can help redefine design problems, making them clearer.
4-Some tools are effective in generating solutions and ideation.

I would like to provide you a name of a design methodology as well as a text about this design methodology.
If you recognize specific tool or concept related to this design methodlogy in the text you should mention the tool and its function.
If you didnot find any tool of the design methodology you must just mention " there is no tool in the text"
In your output please only explain the tools and functions if any tool and function exists in the text and do not mention extra explanation.
Your output should be brief and clear. 
The design Methodology is {design_methodology}.
The Text that you should analyze is: {text}.
In the text you might have some tools, concepts and features related to TRIZ. Please ignore them and only focus on {design_methodology}.

'''
prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
chat_prompt=ChatPromptTemplate.from_messages([prompt])

In [18]:
Human_prompt2= '''
I would like to provide you an analysis about {design_methodology}. 
Within this analysis, you will encounter several sentences akin to the following: "There are no tools available for {design_methodology}."
Please eliminate such sentences and return the original text exactly as it is.
This is the analysis=\n{analysis}
'''
prompt2= HumanMessagePromptTemplate.from_template(Human_prompt2)
chat_prompt2=ChatPromptTemplate.from_messages([prompt2])

In [22]:
analysis=""
design_methodology='OTSM-TRIZ'
for s in splitted_docs:
    for x in s:
        request=chat_prompt.format_prompt(text=x, design_methodology=design_methodology).to_messages()
        chat= ChatOpenAI(openai_api_key=openAI_API_Key)
        result=chat(request)
        analysis=analysis+"\n"+result.content
        print(result.content)

There is no tools in the text.
There is no tools in the text.
The text does not mention any tools related to the OTSM-TRIZ design methodology.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
The text does not mention any tools related to OTSM-TRIZ.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
There is no tools in the text.
The text mentions the following tools of the OTSM-TRIZ d

In [24]:
#Cleaning the analysis
splitted_analysis = split_text_on_concept(analysis, max_length=1000)
for x in splitted_analysis:
    request2=chat_prompt2.format_prompt(analysis=x, design_methodology=design_methodology).to_messages()
    result2=chat(request2)
    print(f" \n{result2.content}\n")

 
The analysis of OTSM-TRIZ does not mention any specific tools related to the design methodology.

 
The design methodology mentioned in the text is OTSM-TRIZ. The text mentions the use of elements from OTSM-TRIZ theory for representing expert knowledge.

 
The text mentions the ENV model from OTSM-TRIZ theory. This tool helps in better understanding design problems by organizing data into a diagram of Elements-Name of the feature-Value.
The text mentions the tool of harmonizing OTSM with tools from classical TRIZ, which helps in combining the strengths of both methodologies for problem-solving.
The text mentions the OAV model, which is a tool in the OTSM-TRIZ methodology. The function of the OAV model is to represent and operate specific knowledge relevant to the initial problem situation, as well as to fit the needs of the problem-solving process.
The text mentions the ENV model, which has a fractal structure that plays a main role in transforming the initial problem description int